In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
# Read the data file
df = pd.read_pickle('100_fingerprints_expanded_dataset.pkl')

In [3]:
# --------------------------------------
# SECOND STAGE MODEL TRAINING
# --------------------------------------
# Split training data into separate sets for right and left hand
dflh = df[df.Hand==0]
dfrh = df[df.Hand==1]

# Left hand training
Xlh = list(dflh["Image"])
ylh = dflh["Identity"]
X_trainlh, X_testlh, y_trainlh, y_testlh = train_test_split(Xlh,ylh,test_size=0.2) # ERROR! Wrong use of training set.
model_lid = RandomForestClassifier() # "lid" short for "left hand, identity"
model_lid.fit(X_trainlh,y_trainlh)

# Right hand training
Xrh = list(dfrh["Image"])
yrh = dfrh["Identity"]
X_trainrh, X_testrh, y_trainrh, y_testrh = train_test_split(Xrh,yrh,test_size=0.2)
model_rid = RandomForestClassifier() # "rid" short for "right hand, identity"
model_rid.fit(X_trainrh,y_trainrh)
# --------------------------------------
# END OF SECOND STAGE MODEL TRAINING
# --------------------------------------

RandomForestClassifier()

In [4]:
# -------------------------------------
# FIRST STAGE MODEL TRAINING
# -------------------------------------
X_train = X_trainlh+X_trainrh
X_test = X_testlh+X_testrh
ytr = pd.concat([y_trainrh,y_trainlh])
yte = pd.concat([y_testrh,y_testlh])
y_train = []
y_test = []

In [5]:
for i in range(len(ytr)):
    j = ytr.index[i]
    y_train.append([df["Hand"][j],j])

In [7]:
for i in range(len(yte)):
    j = yte.index[i]
    y_test.append([df["Hand"][j],j])

In [16]:
y_train_indexfree = [i[0] for i in y_train]

In [11]:
model_hand = RandomForestClassifier()
model_hand.fit(X_train,y_train_indexfree) # Fit the model fo training data
# -------------------------------------
# END OF FIRST STAGE MODEL TRAINING
# -------------------------------------

RandomForestClassifier()

In [13]:
# --------------------------------------
# MODEL IMPLEMENTATION
# --------------------------------------
pred_hand = model_hand.predict(X_test)

X_lh = []
y_lh = []
X_rh = []
y_rh = []

for i in range(len(pred_hand)):
    if pred_hand[i]==0:
        X_lh.append(X_test[i])
        j = y_test[i][1]
        y_lh.append(df["Identity"][j])
    else:
        X_rh.append(X_test[i])
        j = y_test[i][1]
        y_rh.append(df["Identity"][j])
        
pred_lid = model_lid.predict(X_lh)
pred_rid = model_rid.predict(X_rh)
# --------------------------------------
# END OF MODEL IMPLEMENTATION
# --------------------------------------

In [19]:
pred_lid

array([ 54, 284,  86, ..., 462,  77,  54])

In [18]:
# --------------------------------------
# EVALUATION OF RESULTS
# --------------------------------------
# Calculate accuracy
len_lh = len(pred_lid)
len_rh = len(pred_rid)
score_lh = 0
score_rh = 0
score_tot = 0
for i in range(len_lh):
    if pred_lid[i] == y_lh[i]:
        score_lh += 1/len_lh
        score_tot += 1/(len_lh+len_rh)
for i in range(len_rh):
    if pred_rid[i] == y_rh[i]:
        score_rh += 1/len_rh
        score_tot += 1/(len_lh+len_rh)
        
# Print the result
print(score_lh)
print(score_rh)
print(score_tot)
# --------------------------------------
# END OF EVALUATION OF RESULTS
# --------------------------------------

0.019810508182601216
0.019070321811680564
0.019500000000000014
